In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('./data/gapminder.tsv', sep='\t')
df.head(3)

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710


그룹화

In [3]:
# groupby() 메소드를 사용해서 특정 시리즈를 기준으로 데이터를 그룹화할 수 있다.
lifeExp_by_year = df.groupby('year').lifeExp.mean()
lifeExp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [4]:
lifeExp_by_year = df.groupby(['continent', 'year']).lifeExp.mean()
lifeExp_by_year

continent  year
Africa     1952    39.135500
           1957    41.266346
           1962    43.319442
           1967    45.334538
           1972    47.450942
           1977    49.580423
           1982    51.592865
           1987    53.344788
           1992    53.629577
           1997    53.598269
           2002    53.325231
           2007    54.806038
Americas   1952    53.279840
           1957    55.960280
           1962    58.398760
           1967    60.410920
           1972    62.394920
           1977    64.391560
           1982    66.228840
           1987    68.090720
           1992    69.568360
           1997    71.150480
           2002    72.422040
           2007    73.608120
Asia       1952    46.314394
           1957    49.318544
           1962    51.563223
           1967    54.663640
           1972    57.319269
           1977    59.610556
           1982    62.617939
           1987    64.851182
           1992    66.537212
           1997    68.02051

groupby() 메소드와 같이 사용되는 집계 함수

count() => 누락값을 제외한 그룹별 데이터의 개수  
size() => 누락값을 포함한 그룹별 데이터의 개수  
sum() => 누락값을 제외한 그룹별 데이터의 합계  
mean() => 누락값을 제외한 그룹별 데이터의 평균  
max() => 누락값을 제외한 그룹별 데이터의 최대값  
min() => 누락값을 제외한 그룹별 데이터의 최소값  
var() => 누락값을 제외한 그룹별 데이터의 분산  
std() => 누락값을 제외한 그룹별 데이터의 표준편차  
quantile(q=0.25) => 누락값을 제외한 그룹별 데이터의 1사분위수  
quantile(q=0.5) => 누락값을 제외한 그룹별 데이터의 2사분위수  
quantile(q=0.75) => 누락값을 제외한 그룹별 데이터의 3사분위수  
describe() => 누락값을 제외한 그룹별 데이터의 개수, 평균, 표준편차, 1사분위수, 2사분위수, 3사분위수 최대값  
sem() => 누락값을 제외한 그룹별 데이터의 평균의 표준편차  
first() => 그룹별 데이터의 첫 번째 데이터  
last() => 그룹별 데이터의 첫 번째 데이터  
nth() => 그룹별 데이터의 n 번째 데이터  

groupby() 메소드와 같이 사용되는 집계 함수들은 groupby() 메소드 뒤에 '.'을 찍고 실행하면 되지만 사용자 정의 함수는 groupby() 메소드 뒤에 '.'을 찍어서 바로 실행할 수 없고 agg() 메소드를 사용해서 일괄 적용시켜야 한다.

In [5]:
# 숫자 데이터가 저장된 리스트를 인수로 넘겨받아 평균을 계산해서 리턴하는 함수
def my_mean(values):
    n = len(values)
    total = 0
    for value in values:
        total += value
    return total / n

In [6]:
print(my_mean([1, 2, 3, 4, 5]))

3.0


In [7]:
# age_by_year = df.groupby('year').lifeExp.my_mean() # 에러
# 사용자 정의 함수를 groupby() 메소드에서 사용하려면 agg() 메소드의 인수로 ()없이 넘겨주면 된다.
# agg() 메소드는 agg() 메소드를 실행하는 그룹의 데이터가 자동으로 agg() 메소드의 인수로 지정한 사용자 정의 함수의
# 인수로 전달된다.
age_by_year = df.groupby('year').lifeExp.agg(my_mean)
age_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [8]:
# year 별로 계산된 그룹 평균과 전체 평균의 편차를 계산해서 리턴하는 함수
def my_mean_diff(values, global_mean):
    n = len(values)
    total = 0
    for value in values:
        total += value
    mean = total / n # 그룹 평균
    return mean - global_mean

In [9]:
global_mean = df.lifeExp.mean()
print(global_mean)

59.47443936619714


In [10]:
# agg() 메소드가 사용자 정의 함수의 첫 번째 인수로 그룹 데이터를 전달하고 두 번째 이후의 인수는
# agg() 메소드의 인수로 지정해서 넘긴다.
age_mean_year = df.groupby('year').lifeExp.agg(my_mean_diff, global_mean)
age_mean_year

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

In [11]:
age_mean_year = df.groupby(['continent', 'year']).lifeExp.agg(my_mean_diff, global_mean)
age_mean_year

continent  year
Africa     1952   -20.338939
           1957   -18.208093
           1962   -16.154997
           1967   -14.139901
           1972   -12.023497
           1977    -9.894016
           1982    -7.881574
           1987    -6.129651
           1992    -5.844862
           1997    -5.876170
           2002    -6.149209
           2007    -4.668401
Americas   1952    -6.194599
           1957    -3.514159
           1962    -1.075679
           1967     0.936481
           1972     2.920481
           1977     4.917121
           1982     6.754401
           1987     8.616281
           1992    10.093921
           1997    11.676041
           2002    12.947601
           2007    14.133681
Asia       1952   -13.160045
           1957   -10.155895
           1962    -7.911216
           1967    -4.810799
           1972    -2.155170
           1977     0.136117
           1982     3.143500
           1987     5.376742
           1992     7.062773
           1997     8.54607

그룹화한 결과에 넘파이 집계 메소드 실행하기

In [12]:
# 넘파이 집계 메소드를 실행하려면 딕셔너리에 계산에 사용할 시리즈 이름을 'key'로 실행할 메소드 이름을
# 'value'로 묶어서 전달한다.
numpy_group = df.groupby('year').agg({'lifeExp': 'sum', 'pop': 'mean', 'gdpPercap': 'median'})
numpy_group

,lifeExp,pop,gdpPercap
year,,,
1952,6966.18200,1.695040e+07,1968.528344
1957,7314.05096,1.876341e+07,2173.220291
1962,7612.51336,2.042101e+07,2335.439533
1967,7906.31712,2.265830e+07,2678.334740
1972,8185.92888,2.518998e+07,3339.129407
1977,8458.96236,2.767638e+07,3798.609244
1982,8737.71400,3.020730e+07,4216.228428
1987,8976.19100,3.303857e+07,4280.300366
1992,9110.76800,3.599092e+07,4386.085502


표준 점수 계산하기

데이터의 평균과 표준편차의 차이를 표준 점수라 부른다.  
표준 점수의 표준편차를 계산하면 변환된 데이터의 평균이 0이 되고 표준편차가 1이 된다. 그러면 데이터가 표준화되어 서로 다른 데이터를 쉽게 비교할 수 있다.

agg() 메소드는 그룹화된 데이터를 사용자 정의 함수에 전달해서 그룹별 대표값 1개를 만들지만 transform() 메소드는 그룹화된 데이터를 사용자 정의 함수에 전달해서 그룹별 데이터 각각에 대한 결과를 만든다.

In [13]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [14]:
df['agg_mean'] = df.groupby('year').lifeExp.agg(my_mean)
df['transform_zscore'] = df.groupby('year').lifeExp.transform(my_zscore)
df

,country,continent,year,lifeExp,pop,gdpPercap,agg_mean,transform_zscore
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,NaN,-1.656854
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,NaN,-1.731249
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,NaN,-1.786543
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,NaN,-1.848157
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,NaN,-1.894173
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,NaN,-0.081621
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,NaN,-0.336974
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,NaN,-1.574962
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,NaN,-2.093346


누락값을 평균값으로 처리하기

In [15]:
# seaborn 라이브러리의 tips 데이터 셋에서 무작위로 10개의 행 데이터를 가져온 다음 total_bill 열의 값 4개를
# 임의로 선택해서 누락값으로 바꾼다.
# 매번 실행할 때 마다 같은 배열의 데이터를 추출하기 위해 seed() 메소드를 사용해서 난수를 고정한다.
np.random.seed(42)
# sample() 메소드는 인수로 지정한 개수만큼 전체 데이터에서 데이터를 랜덤하게 추출한다.
tips_df = sns.load_dataset('tips').sample(10)
tips_df

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


넘파이의 permutation() 메소드와 shuffle() 메소드는 무작위로 배열을 섞어주는 공통된 기능을 실행하지만 차이점은 permutation() 메소드는 섞은 결과를 리턴해서 원본 자체는 수정되지 않지만 shuffle() 메소드는 섞은 결과로 원본 자체를 수정한다.

x = np.arange(10)  
print(x) # [0 1 2 3 4 5 6 7 8 9]이 출력된다. => 원본  
print(np.random.permutation(x)) # 섞인 결과가 출력된다.  
print(x) # [0 1 2 3 4 5 6 7 8 9]이 출력된다. => permutation() 메소드는 원본을 변형시키지 않는다.  
print(np.random.shuffle(x)) # None  
print(x) # shuffle() 메소드에 의해 섞인 결과가 출력된다. => shuffle() 메소드는 원본을 변형한다.

In [16]:
tips_df.loc[np.random.permutation(tips_df.index)[:4], 'total_bill'] = np.NaN
tips_df

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [32]:
# total_bill 열의 누락값을 total_bill 열 전체 평균으로 채우기
tips_df.total_bill.fillna(tips_df.total_bill.mean())

24     19.820000
6       8.770000
153    16.611667
211    16.611667
198    16.611667
176    16.611667
192    28.440000
124    12.480000
9      14.780000
101    15.380000
Name: total_bill, dtype: float64

In [23]:
# tips_df의 데이터는 여성보다 남성이 더 많기 때문에 여성과 남성을 구분해서 total_bill 열의 평균을 계산하지
# 않으면 여성 데이터가 남성 데이터의 영향(간섭)을 받아서 여성 데이터가 회손될 수 있다.
count_sex = tips_df.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [25]:
print('total_bill 전체 평균: {}'.format(tips_df.total_bill.mean()))
print(tips_df.groupby('sex').mean())

total_bill 전체 평균: 16.611666666666668
        total_bill       tip      size
sex                                   
Male       17.9525  2.875714  2.571429
Female     13.9300  2.506667  2.000000


In [28]:
# 성별별로 그룹화해서 total_bill 열의 데이터를 받아 평균을 계산해서 성별별 평균으로 누락값을
# 수정하는 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [30]:
total_bill_group_mean = tips_df.groupby('sex').total_bill.transform(fill_na_mean)
total_bill_group_mean

24     19.8200
6       8.7700
153    17.9525
211    17.9525
198    13.9300
176    17.9525
192    28.4400
124    12.4800
9      14.7800
101    15.3800
Name: total_bill, dtype: float64